In [ ]:
%matplotlib qt
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.optim as optim

%load_ext autoreload
%autoreload 2

In [ ]:
#from forward import *
import sys
sys.path.append("../static/")
sys.path.append("..")
import kinematics
from inverse import with_torch
from presets_robot_models import preset_models
#from learn import evaluate_plot, train_motors
from learn import *

In [ ]:
alpha, theta, radius, dists, active, limits = get_DH_params(preset_models["HangingArm"])

In [ ]:
For_model = kinematics.Kinematics("HangingArm")

In [ ]:
model = torch.load("./kine_models/HangingArm.pt")

# For Training

In [ ]:
For_model.orientation = False
inputs, target = For_model.generate_maps()
n_in  = inputs.shape[1]
n_out = target.shape[1]
model = NeuralNetworkStack(n_in, n_out, nlayers=5, depth=20)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# generate map

In [ ]:
inputs, target = For_model.generate_maps()

In [ ]:
For_model.orientation = False

In [ ]:
evaluate_plot(For_model, model)

In [ ]:
train_positions(For_model, model)

In [ ]:
train_motors(For_model, model)

# Blind Testing

In [ ]:
For_model.orientation = True
inputs, targets = For_model.generate_paths()
model = NeuralNetworkBlind(inputs.shape[1], targets.shape[1], nlayers=8, depth=10)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
train_positions_paths(For_model, model, 2000)

In [126]:
evaluate_plot(For_model, model)

tensor([[ -6., 135., 199., 360., 180.,  88.],
        [ -2., 135., 198., 359., 180.,  89.],
        [  0., 135., 203.,   2., 180.,  90.],
        [  2., 134., 198., 359., 180.,  91.],
        [ -6., 134., 199., 359., 180.,  88.]])
tensor(13404.8438, grad_fn=<MeanBackward0>)
tensor(176.7262)


# Prototype code to test Machine

In [ ]:
inputs, target = For_model.generate_maps(n=1)
#mot_pos = np.array([alpha,theta,radius,dists])
n= np.random.randint(len(inputs))
#######################
pred = model(torch.Tensor(inputs[n:n+1])).detach()
######################
For_model.set_active(pred[0])
For_model.update_position()
pos_pred = For_model.run_forward().round(2)
#######################
For_model.set_active(target[n])
pos_targ = For_model.run_forward().round(2)
########################
print("--- x y z ---")
print(inputs[n].round(2))
print(pos_targ)
print(pos_pred )
print("--- torch ---")
pred_xys = with_torch().forward_from_active(For_model, target[n:n+1])
print(pred_xys[0,0].numpy().round(2))
pred_xys = with_torch().forward_from_active(For_model, pred)
print(pred_xys[0,0].numpy().round(2))
print("--- motor positions ---")
print(target[n].round(2))
print(pred.numpy().round(2)[0])

In [ ]:
def get_DH(For_model):
    alpha  = torch.tensor(1.*np.array(For_model.alpha)) 
    theta  = torch.tensor(1.*np.array(For_model.theta)) 
    radius = torch.tensor(1.*np.array(For_model.radius))
    dists  = torch.tensor(1.*np.array(For_model.dists)) 
    active = For_model.active
    DH = torch.stack([alpha, theta, radius, dists])
    return DH

def get_transform(DH):
    alpha, theta, radius, dists = DH
    alpha = torch.deg2rad(alpha)
    theta = torch.deg2rad(theta)
    
    TFs = make_transforms(alpha, theta, radius, dists)
    return TFs[-1]

def make_transforms(alpha, theta, radius, dists):

    mot_params = torch.stack([alpha, theta, radius, dists])

    Tfs = []
    T = torch.eye(4)
    for i in range(mot_params.shape[1]):
        a,t,r,d = mot_params[:,i]
        DHt = DH_transform(a,t,r,d) 
        T = torch.matmul(T,DHt)     
        Tfs.append(T)

    return Tfs

def DH_transform(a,t,r,z):
        Zt = Z_transform(a,t,r,z)
        Xt = X_transform(a,t,r,z)
        DHt = torch.matmul(Zt,Xt)
        return DHt

    ###############################################
def X_transform(a,t,r,d):

    Xt = torch.eye(4)
    Xt[0,3] = r
    Xt[1,1] = torch.cos(a)
    Xt[1,2] = -1*torch.sin(a)
    Xt[2,1] = torch.sin(a)
    Xt[2,2] = torch.cos(a)

    return Xt

def Z_transform(a,t,r,d):

    Zt = torch.eye(4)
    Zt[0,0] = torch.cos(t)
    Zt[0,1] = -1*torch.sin(t)
    Zt[1,0] = torch.sin(t)
    Zt[1,1] = torch.cos(t)
    Zt[2,3] = d

    return Zt

In [ ]:
matrix = torch.Tensor(np.eye(3))

In [ ]:
rot_angle = torch_matrix_to_rotation( matrix )
rot_angle

In [ ]:
DH = get_DH(For_model)
TF = get_transform(DH)
matrix = TF[:3,:3]

In [ ]:
rotation = np.array(rotation)

In [ ]:
DH = get_DH(For_model)
TF = get_transform(DH)
matrix = TF[:3,:3]

matrix = torch.Tensor(R.from_euler("XYZ",[np.pi/2,np.pi/3,np.pi]).as_matrix())

In [ ]:
for _ in range(100):
    rot = np.random.uniform(np.pi,size=(3))
    out = R.from_matrix(R.from_euler("XYZ",rot).as_matrix()).as_euler("XYZ")
    if out[0]<0:  
        out[[0,2]] = out[[0,2]] + np.pi
        out[[1]]   = np.pi - out[[1]] 
    print(rot, out, rot.round(4) == out.round(4)) 

In [ ]:
For_model = kinematics(alpha, theta, radius, dists, active, limits)

In [ ]:
inputs, target = For_model.generate_maps()
#mot_pos = np.array([alpha,theta,radius,dists])
#n= np.random.randint(len(inputs))
pred = model(torch.Tensor(inputs)).detach().numpy()

In [ ]:
For_model = kinematics(alpha, theta, radius, dists, active, limits)

In [ ]:
For_model.orientation = True

## 2D TEST

In [ ]:
size = 1000
t = np.random.randint(-10,370,size=size)
r = np.random.randint(100,size=size)
inputs = np.array([r,t]).T

x = np.sin(np.radians(t))*r
y = np.cos(np.radians(t))*r
target = np.array([x,y]).T

inputs,target = target,inputs

In [ ]:
n_in  = inputs.shape[1]
n_out = target.shape[1]
model = NeuralNetworkStack(n_in, n_out, nlayers=15, depth=100)
#model = NeuralNetworkSinusoid(n_in, n_out)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
#optimizer = optim.Adam(model.parameters(), lr=1e-3)
for n in range(100):
    size = 500
    t = np.random.randint(-10,370,size=size)
    r = np.random.randint(150,size=size)
    inputs = np.array([r,t]).T

    x = np.sin(np.radians(t))*r
    y = np.cos(np.radians(t))*r
    target = np.array([x,y]).T
    
    train_loop(model, inputs, target, distance_loss,optimizer)
        
    if n % 10 == 0:
        optimizer.param_groups[0]["lr"] = optimizer.param_groups[0]["lr"]*0.5        

In [ ]:
n= np.random.randint(len(inputs))
xy = target[n]
pred = model(torch.Tensor(inputs[n:n+1])).detach().numpy()
print(inputs[n])
print(target[n])
print(pred.round(2))

In [ ]:
size = 1000
t = np.random.randint(-10,370,size=size)
r = np.random.randint(150,size=size)
inputs = np.array([r,t]).T

x = np.sin(np.radians(t))*r
y = np.cos(np.radians(t))*r
target = np.array([x,y]).T

inputs, target = target, inputs

pred = model(torch.Tensor(inputs)).detach().numpy()



r = inputs[:,0]
t = inputs[:,1]

x = target[:,0]
y = target[:,1]

x1 = r*np.sin(np.radians(t))
y1 = r*np.cos(np.radians(t))

x2 = pred[:,0]
y2 = pred[:,1]

x2 = r*np.sin(np.radians(x2))
y2 = r*np.cos(np.radians(y2))

plt.plot(x,y,"bo")
#plt.plot(x1,y1,)
plt.plot(x2,y2,"ro")
plt.plot([x,x2],[y,y2],"g")

In [ ]:
size = 1000

pred = model(torch.Tensor(inputs)).detach().numpy()

x = target[:,0]
y = target[:,1]

x2 = pred[:,0]
y2 = pred[:,1]


plt.plot(x,y,"bo")
#plt.plot(x1,y1,)
plt.plot(x2,y2,"ro")
plt.plot([x,x2],[y,y2],"g")

In [ ]:
    def forward(self,x_in):
                
        x_next = self.layer_first(x_in)
    
        out_list = [x_in,x_next]
        for n, block in enumerate(self.blocks):
            x_next = block(x_next) 
            out_list.append(x_next)  
            
        x = torch.cat(out_list, dim=1)
        
        x = F.leaky_relu(x)
        print(x)

        out = self.layer_last(x)
        
        return out